In [1]:
# Import du paquet PythonMIP et de toutes ses fonctionnalités
from mip import *
# Import du paquet time pour calculer le temps de résolution
import time 
import InstanceReader

In [18]:
def solve(data, L, K):
    print("in process...")
    # Création du modèle vide 
    model = Model(name = "MTZ-EAF", sense = mip.MAXIMIZE, solver_name="CBC")
    
    #### Création des variables y, u et t ####
    y=[]
    u=[]
    for i in range(data.id_max+1):
        y.append([])
        for j in range(data.id_max+1):
            y[i].append(model.add_var(name="y(" + str(i) + "," + str(j) + ")", var_type=BINARY))

    for k in range(data.id_max+1):
        u.append([])
        for i in range(data.id_max+1):
            u[k].append([])
            for j in range(data.id_max+1):
                u[k][i].append(model.add_var(name="u("+str(k) + "," + str(i) + "," + str(j)+ ")", var_type=BINARY))
    
    t = [model.add_var(name="t(" + str(i)  + ")", lb=0, ub=(L-1) , var_type=CONTINUOUS) for i in range(data.id_max+1)]
    #On a fait la contrainte 21 : ti <= L-1 pour tout i de V    
    
    
    #### fonction objectif ####
    model += (xsum(data.cost[i][j] * y[i][j]  for i in (data.V) for j in (data.allPatients[int(data.index[int(i)])]))
             + xsum(xsum(data.cost[i][j]*u[k][i][j] for i in (data.V) for j in (data.allPatients[int(data.index[int(i)])])) for k in (data.P)))
    
    
    #### Les contraintes ####
    
    # Contrainte 17
    for k in (data.P):
        for i in (data.P):
            model += (xsum(u[k][i][j] for j in data.allPatients[int(data.index[int(i)])])
                      -xsum(u[k][int(data.allDonorsInP[i][j])][i] for j in range(1,int(data.allDonorsInP[i][0])+1)) == 0 ) 
    
    #Contrainte 18
    for i in (data.P):
        model += (xsum(y[i][j] for j in data.allPatients[int(data.index[int(i)])])
                      -xsum(y[int(data.allDonors[i][j])][i] for j in range(1,int(data.allDonors[i][0])+1)) == 0 ) 
    
    #Contrainte 19
    for i in (data.P):
        model += (xsum(y[i][j] for j in data.allPatients[int(data.index[int(i)])])
                  + xsum(xsum(u[k][i][j] for j in data.allPatients[int(data.index[int(i)])]) for k in (data.P)) <= 1)
    
    #Contrainte 20
    for i in (data.N):
        model += (xsum(y[i][j] for j in data.allPatients[int(data.index[int(i)])]) <= 1 )
        
    #Contrainte 21
    for k in (data.P):
        model += (xsum(u[k][i][j] for i in (data.V) for j in (data.allPatients[int(data.index[int(i)])])) <= K)
        
    #Contrainte 23
    for i in (data.P):
        for k in (data.P):
            if(k>=i):
                break
            model += (xsum(u[k][i][j] for j in (data.allPatients[int(data.index[int(i)])]))
                      - xsum(u[k][k][j] for j in (data.allPatients[int(data.index[int(k)])])) <= 0)
            
    #Contrainte 24
    for i in (data.P):
        for k in (data.P):
            if(k<=i):
                continue
            model += (xsum(u[k][i][j] for j in (data.allPatients[int(data.index[int(i)])])) == 0)
    
    #CONTRAINTE NON EXISTENTE DANS LE MODELE
    #for i in (data.N):
        #for k in (data.P): 
            #model += (xsum(u[k][i][j] for j in (data.allPatients[int(data.index[int(i)])])) == 0)
            
    #Contrainte 25
    # !!! ne fonctionne pas  !!!
    #for i in (data.V):
        #for j in (data.allPatients[int(data.index[int(i)])]):
            #model += (t[i] - t[j] + (len(data.P)+2) * y[i][j] <= len(data.P)+1)
            
    #Contrainte 26:
    #for i in (data.N):
     #   model += (t[i]==0)
                  
    # Limitation du nombre de processeurs
    model.threads = 1  
    # Lancement du chronomètre
    start = time.perf_counter()
    # Résolution du modèle
    status = model.optimize(max_seconds=120)  # temps limite = 120s
    # Arrêt du chronomètre et calcul du temps de résolution
    runtime = time.perf_counter() - start
    
    print("\n----------------------------------")
    if status == OptimizationStatus.OPTIMAL:
        print("Status de la résolution: OPTIMAL")
    elif status == OptimizationStatus.FEASIBLE:
        print("Status de la résolution: TEMPS LIMITE et SOLUTION REALISABLE CALCULEE")
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print("Status de la résolution: TEMPS LIMITE et AUCUNE SOLUTION CALCULEE")
    elif status == OptimizationStatus.INFEASIBLE or status == OptimizationStatus.INT_INFEASIBLE:
        print("Status de la résolution: IRREALISABLE")
    elif status == OptimizationStatus.UNBOUNDED:
        print("Status de la résolution: NON BORNE")
        
    print("Temps de résolution (s) : ", runtime)    
    print("----------------------------------")

    # Si le modèle a été résolu à l'optimalité ou si une solution a été trouvée dans le temps limite accordé
    if model.num_solutions>0:
        print(data.P)
        print("Solution calculée")
        print("-> Valeur de la fonction objectif de la solution calculée : ",  model.objective_value)  # ne pas oublier d'arrondir si le coût doit être entier
        print("-> Meilleure borne inférieure sur la valeur de la fonction objectif = ", model.objective_bound)
        for i in range(data.id_max+1):
            for j in range(data.id_max+1):
                if(y[i][j].x>0.5):
                    print("(chemin) le donneur " + str(i) + " donne son rein au patient " + str(j) + " en position " + str(t[i].x))
        for k in range(data.id_max+1):
            for i in range(data.id_max+1):
                for j in range(data.id_max+1):
                    if(u[k][i][j].x>0.5):
                        print("(cycle) le donneur " + str(i) + " donne son rein au patient " + str(j) + " dans le cycle " + str(k))
        donneur=[]
        for i in (data.V):
            test=0 
            for j in (data.allPatients[int(data.index[int(i)])]):
                if(y[i][j].x>0.5):
                    test=1
            for k in range (data.id_max+1):       
                for j in (data.allPatients[int(data.index[int(i)])]):
                    if(u[k][i][j].x>0.5):
                        test=1
            if(test==0):
                donneur.append(i)
        
        print("liste des donneurs non utilisé " + str(donneur))
        patient=[]
        for i in (data.P):
            test=0 
            for j in range(1,int(data.allDonorsInP[i][0])+1):
                if(y[int(data.allDonorsInP[i][j])][i].x>0.5):
                    test=1
            for k in range (data.id_max+1):       
                for j in range(1,int(data.allDonorsInP[i][0])+1):
                    if(u[k][int(data.allDonorsInP[i][j])][i].x>0.5):
                        test=1
            if(test==0):
                patient.append(i)
        print("liste des patients sans donneurs " + str(patient))
        
        
    

In [14]:
solve(InstanceReader.Data(str(50),str(0)), 10, 10)

in process...

----------------------------------
Status de la résolution: OPTIMAL
Temps de résolution (s) :  44.827739384999404
----------------------------------
[0, 1, 2, 9, 10, 11, 19, 20, 21, 23, 24, 26, 35, 38, 40, 44, 45, 46, 47, 50, 51, 52, 53, 54, 57, 62, 64, 67, 68, 70, 71, 74, 75, 85, 86, 90, 97, 100, 101, 102, 104, 108, 110, 111, 113, 114, 115, 119, 120, 121]
Solution calculée
-> Valeur de la fonction objectif de la solution calculée :  1860.0
-> Meilleure borne inférieure sur la valeur de la fonction objectif =  1860.0
(chemin) le donneur 0 donne son rein au patient 111 en position 0.0
(chemin) le donneur 11 donne son rein au patient 19 en position 0.0
(chemin) le donneur 19 donne son rein au patient 54 en position 0.0
(chemin) le donneur 21 donne son rein au patient 47 en position 0.0
(chemin) le donneur 26 donne son rein au patient 21 en position 0.0
(chemin) le donneur 44 donne son rein au patient 0 en position 0.0
(chemin) le donneur 46 donne son rein au patient 53 en 

In [19]:
solve(InstanceReader.Data(str(50),str(0)), 10, 10)

in process...

----------------------------------
Status de la résolution: OPTIMAL
Temps de résolution (s) :  10.414641073999519
----------------------------------
[0, 1, 2, 9, 10, 11, 19, 20, 21, 23, 24, 26, 35, 38, 40, 44, 45, 46, 47, 50, 51, 52, 53, 54, 57, 62, 64, 67, 68, 70, 71, 74, 75, 85, 86, 90, 97, 100, 101, 102, 104, 108, 110, 111, 113, 114, 115, 119, 120, 121]
Solution calculée
-> Valeur de la fonction objectif de la solution calculée :  44610.0
-> Meilleure borne inférieure sur la valeur de la fonction objectif =  44610.0
(chemin) le donneur 0 donne son rein au patient 111 en position 0.0
(chemin) le donneur 10 donne son rein au patient 68 en position 0.0
(chemin) le donneur 11 donne son rein au patient 19 en position 0.0
(chemin) le donneur 19 donne son rein au patient 54 en position 0.0
(chemin) le donneur 21 donne son rein au patient 47 en position 0.0
(chemin) le donneur 24 donne son rein au patient 10 en position 0.0
(chemin) le donneur 26 donne son rein au patient 21 